In [3]:
# client_id = "---"
# client_secret = "---"

## 네이버 쇼핑 검색 api 활용

In [5]:
## query 입력받고 api호출 후 바로 df로 변환하여 저장해놓기

import urllib.request
import urllib.parse
import pandas as pd
import json

# 클라이언트 ID와 시크릿
client_id = "---"  
client_secret = "---"

# 사용자로부터 검색어 입력 받기
query = input("search: ")
encoded_query = urllib.parse.quote(query)

# API 요청 URL 설정
url = f"https://openapi.naver.com/v1/search/shop?query={encoded_query}&display=100"
request = urllib.request.Request(url)

# 헤더에 클라이언트 ID와 시크릿 추가
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)

# API 요청 후 응답 받기
response = urllib.request.urlopen(request)
response_body = response.read().decode('utf-8')  

# 응답 데이터를 JSON 형식으로 파싱
data = json.loads(response_body)

data

# 데이터 프레임 생성
df = pd.DataFrame(data['items'])

# 검색어 컬럼 추가
df['query'] = query  # 사용자가 입력한 검색어를 컬럼에 추가

# HTML 태그 제거 (옵션)
df['title'] = df['title'].str.replace('<[^<]+?>', '', regex=True)

# 결과 확인
df.head()

search:  핸드크림


{'lastBuildDate': 'Sat, 20 Apr 2024 13:02:27 +0900',
 'total': 513077,
 'start': 1,
 'display': 100,
 'items': [{'title': '록시땅 시어 드라이 스킨 <b>핸드 크림</b> 150ml',
   'link': 'https://search.shopping.naver.com/gate.nhn?id=13155402252',
   'image': 'https://shopping-phinf.pstatic.net/main_1315540/13155402252.20220426090316.jpg',
   'lprice': '21600',
   'hprice': '',
   'mallName': '네이버',
   'productId': '13155402252',
   'productType': '1',
   'brand': '록시땅',
   'maker': '',
   'category1': '화장품/미용',
   'category2': '바디케어',
   'category3': '핸드케어',
   'category4': ''},
  {'title': '탬버린즈 퍼퓸 쉘 엑스 <b>핸드크림</b> 카모 30ml',
   'link': 'https://search.shopping.naver.com/gate.nhn?id=35706026619',
   'image': 'https://shopping-phinf.pstatic.net/main_3570602/35706026619.20221108103424.jpg',
   'lprice': '27290',
   'hprice': '',
   'mallName': '네이버',
   'productId': '35706026619',
   'productType': '1',
   'brand': '탬버린즈',
   'maker': '정코스',
   'category1': '화장품/미용',
   'category2': '바디케어',
   'category3

In [6]:
# 20240420 update
# productId가 "8"로 시작하는 것은 가격비교x 바로 구매 쇼핑몰 사이트로 연결됨
# > 스크래핑이 용이하지 않고, 가격비교가 아니므로 필터링하기로..
# 최대 100개 리스트 요청 가능 / productId가 8로 시작하는것 제외하고 100개 항목 채우기 (가능 )

import urllib.request
import urllib.parse
import pandas as pd
import json

# 클라이언트 ID와 시크릿
client_id = "---"  
client_secret = "---"

# 사용자로부터 검색어 입력 받기
query = input("search: ")
encoded_query = urllib.parse.quote(query)

# 초기 설정
url_base = "https://openapi.naver.com/v1/search/shop"
items_collected = []
start_index = 1
max_results = 100
desired_count = 100  # 원하는 아이템 수

while len(items_collected) < desired_count:
    # API 요청 URL 설정
    url = f"{url_base}?query={encoded_query}&display=100&start={start_index}"
    request = urllib.request.Request(url)

    # 헤더에 클라이언트 ID와 시크릿 추가
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    # API 요청 후 응답 받기
    response = urllib.request.urlopen(request)
    response_body = response.read().decode('utf-8')  

    # 응답 데이터를 JSON 형식으로 파싱
    data = json.loads(response_body)
    
    # productId가 8로 시작하는 아이템 필터링
    filtered_items = [item for item in data['items'] if not item['productId'].startswith('8')]
    items_collected.extend(filtered_items)
    
    # 가져올 수 있는 아이템이 더 이상 없거나 충분한 수를 얻은 경우 반복 종료
    if len(data['items']) < 100 or len(items_collected) >= desired_count:
        break

    # 다음 페이지 인덱스 업데이트
    start_index += 100

# 데이터 프레임 생성
df = pd.DataFrame(items_collected[:desired_count])

# 검색어 컬럼 추가
df['query'] = query  # 사용자가 입력한 검색어를 컬럼에 추가

# HTML 태그 제거 (옵션)
df['title'] = df['title'].str.replace('<[^<]+?>', '', regex=True)

# 결과 확인
print(df.head())


search:  핸드크림


                       title  \
0  록시땅 시어 드라이 스킨 핸드 크림 150ml   
1  탬버린즈 퍼퓸 쉘 엑스 핸드크림 카모 30ml   
2    카밀 핸드 앤 네일 클래식 크림 100ml   
3           카밀 핸드크림 미니 5종 세트   
4        록시땅 시어 버터 핸드 크림 30g   

                                                link  \
0  https://search.shopping.naver.com/gate.nhn?id=...   
1  https://search.shopping.naver.com/gate.nhn?id=...   
2  https://search.shopping.naver.com/gate.nhn?id=...   
3  https://search.shopping.naver.com/gate.nhn?id=...   
4  https://search.shopping.naver.com/gate.nhn?id=...   

                                               image lprice hprice mallName  \
0  https://shopping-phinf.pstatic.net/main_131554...  21600             네이버   
1  https://shopping-phinf.pstatic.net/main_357060...  27290             네이버   
2  https://shopping-phinf.pstatic.net/main_679923...   3580             네이버   
3  https://shopping-phinf.pstatic.net/main_299352...   8400             네이버   
4  https://shopping-phinf.pstatic.net/main_547823...  11720             네이버

In [7]:
df

,title,link,image,lprice,hprice,mallName,productId,productType,brand,maker,category1,category2,category3,category4,query
0,록시땅 시어 드라이 스킨 핸드 크림 150ml,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_131554...,21600,,네이버,13155402252,1,록시땅,,화장품/미용,바디케어,핸드케어,,핸드크림
1,탬버린즈 퍼퓸 쉘 엑스 핸드크림 카모 30ml,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_357060...,27290,,네이버,35706026619,1,탬버린즈,정코스,화장품/미용,바디케어,핸드케어,,핸드크림
2,카밀 핸드 앤 네일 클래식 크림 100ml,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_679923...,3580,,네이버,6799239096,1,카밀,카밀,화장품/미용,바디케어,핸드케어,,핸드크림
3,카밀 핸드크림 미니 5종 세트,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_299352...,8400,,네이버,29935250621,1,카밀,,화장품/미용,바디케어,핸드케어,,핸드크림
4,록시땅 시어 버터 핸드 크림 30g,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_547823...,11720,,네이버,5478237110,1,록시땅,록시땅,화장품/미용,바디케어,핸드케어,,핸드크림
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,허바신 카밀 핸드크림 75ml,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_216084...,7590,,네이버,21608497541,1,허바신,,화장품/미용,바디케어,핸드케어,,핸드크림
96,탬버린즈 핸드 크림 7 65ml,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_278619...,32000,,네이버,27861998533,1,탬버린즈,코스맥스,화장품/미용,바디케어,핸드케어,,핸드크림
97,아트릭스 핸드크림 400ml 2개 핸드 앤 네일 케어 대용량 크림 로션,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_124662...,11900,,메이크업아이,12466246568,2,아트릭스,,화장품/미용,바디케어,핸드케어,,핸드크림
98,바이레도 집시워터 핸드크림 30ml,https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_107367...,44000,,네이버,10736795244,1,바이레도,신세계인터네셔날,화장품/미용,바디케어,핸드케어,,핸드크림


In [28]:
# # csv 저장
# csv_file_path = 'C:/Users/kibwa/Desktop/ftp/navershopping.csv'
# df.to_csv (csv_file_path, index=False, encoding='utf-8')

#json 저장
json_file_path = 'C:/Users/kibwa/Desktop/ftp/navershopping.json'
df.to_json (json_file_path, index=False)

### 링크에 따른 리뷰 수집 자동화
가설 ) <br>
api로 받아온 link 속성을 추출해서 selenium으로 접근한 다음 > 사러가기 버튼 클릭 > 메뉴바의 리뷰 탭의 컨텐츠를 스크래핑하기
:: (240420) link 데이터 접속이 막혀있음 "부적절한 요청"

해결 240420 ) <br>
api로 productId와 query속성을 추출해서 url을 조합하기 <br>
https://search.shopping.naver.com/catalog/{productId}?query={query_string} 형태 <br>

In [10]:
# 20240420 update

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

# WebDriver 초기화
driver = webdriver.Chrome()

# 스크롤 다운 함수
def scroll_down(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 페이지 로드를 위한 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 스크롤 높이가 변하지 않았다면 중단
            break
        last_height = new_height

# 리뷰 수집 함수
def collect_reviews(url):
    try:
        driver.get(url)
        
        # "쇼핑몰리뷰" 탭 클릭
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.floatingTab_on__2FzR0 a[aria-selected="true"] strong'))
        )
        review_tab = driver.find_element(By.CSS_SELECTOR, 'li.floatingTab_on__2FzR0 a[aria-selected="true"] strong')
        if review_tab.text == "쇼핑몰리뷰":
            review_tab.click()
        
        # 스크롤 다운하여 리뷰 로드
        scroll_down(driver)
        
        # 리뷰 내용 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = [p.text for p in soup.select('div.reviewItems_review__DqLYb div.reviewItems_review_text__dq0kE p.reviewItems_text__XrSSf')]
        return reviews
    except Exception as e:
        print(f"Failed to process URL {url}: {e}")
        return []

# DataFrame 순회 및 URL 생성
all_reviews = {}
for index, row in df.iterrows():
    product_id = row['productId']
    query_string = urllib.parse.quote(row['query'])
    url = f"https://search.shopping.naver.com/catalog/{product_id}?query={query_string}"
    
    # 리뷰 수집
    reviews = collect_reviews(url)
    all_reviews[product_id] = reviews if reviews else ["No reviews or page not found"]

# 결과 출력
for product_id, reviews in all_reviews.items():
    print(f"Product ID: {product_id}, Collected Reviews: {reviews}")

# WebDriver 종료
driver.quit()




Failed to process URL https://search.shopping.naver.com/catalog/12582301973?query=%ED%95%B8%EB%93%9C%ED%81%AC%EB%A6%BC: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF78D38F922+3045234]
	(No symbol) [0x00007FF78D033652]
	(No symbol) [0x00007FF78CEF7564]
	(No symbol) [0x00007FF78CF45F9D]
	(No symbol) [0x00007FF78CF4606C]
	(No symbol) [0x00007FF78CF8BDD7]
	(No symbol) [0x00007FF78CF69DEF]
	(No symbol) [0x00007FF78CF891C4]
	(No symbol) [0x00007FF78CF69B53]
	(No symbol) [0x00007FF78CF3813C]
	(No symbol) [0x00007FF78CF38D71]
	GetHandleVerifier [0x00007FF78D3BE89D+3237613]
	GetHandleVerifier [0x00007FF78D400BA7+3508727]
	GetHandleVerifier [0x00007FF78D3F976F+3478975]
	GetHandleVerifier [0x00007FF78D16F1C6+815126]
	(No symbol) [0x00007FF78D03ED0F]
	(No symbol) [0x00007FF78D039854]
	(No symbol) [0x00007FF78D0399E2]
	(No symbol) [0x00007FF78D02A574]
	BaseThreadInitThunk [0x00007FFB36737344+20]
	RtlUserThreadStart [0x00007FFB377826B1+33]

Failed to process URL https://search.shopping.naver.c

In [16]:
# 결과 및 리뷰 개수 출력
# 리뷰 파싱은 일단 1페이지(리뷰개수 productId 당 최대 20개)만 했음
# 리뷰를 어떻게 보여줄지 / 리뷰 분석을 해야할지 / 리뷰 텍스트를 어떤식으로 활용할지는 조금 더 고민해보기...
line_count = 0 
for product_id, reviews in all_reviews.items():
    print(f"Product ID: {product_id}, Number of Reviews Collected: {len(reviews)}")
    line_count += 1 

print(f"Total number of lines printed: {line_count}")

Product ID: 13155402252, Number of Reviews Collected: 20
Product ID: 35706026619, Number of Reviews Collected: 20
Product ID: 6799239096, Number of Reviews Collected: 20
Product ID: 29935250621, Number of Reviews Collected: 20
Product ID: 5478237110, Number of Reviews Collected: 20
Product ID: 17279363154, Number of Reviews Collected: 20
Product ID: 13246584152, Number of Reviews Collected: 20
Product ID: 22446328494, Number of Reviews Collected: 20
Product ID: 45719313621, Number of Reviews Collected: 20
Product ID: 29519677619, Number of Reviews Collected: 20
Product ID: 9197591043, Number of Reviews Collected: 20
Product ID: 17167523500, Number of Reviews Collected: 20
Product ID: 22444963903, Number of Reviews Collected: 20
Product ID: 9279134189, Number of Reviews Collected: 20
Product ID: 32486820622, Number of Reviews Collected: 20
Product ID: 13038713845, Number of Reviews Collected: 20
Product ID: 21300766930, Number of Reviews Collected: 20
Product ID: 32864886625, Number of 

### 최저가 링크 수집(자동화x)

작성자 : 장다은<br>
작성일 : 20240416

selenium 이용,
url : 네이버 쇼핑 "핸드크림" 검색 > "탬버린즈 퍼퓸 쉘 엑스 핸드크림 카모 30ml" 가격비교 창에서 최저가 링크 수집

In [33]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

# WebDriver 초기화
driver = webdriver.Chrome()

# 스크롤 다운 함수
def scroll_down(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 페이지 로드를 위한 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 스크롤 높이가 변하지 않았다면 중단
            break
        last_height = new_height

# # 링크 수집 함수
# def collect_links(url):
#     driver.get(url)
#     scroll_down(driver)  # 페이지를 맨 아래까지 스크롤 다운
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     links = [urljoin(url, a['href']) for a in soup.find_all('a', class_='productList_title__R1qZP linkAnchor') if a.get('href')]
#     return links

# 최저가 링크 수집 함수
def collect_lowest_price_link(url):
    driver.get(url)
    scroll_down(driver)  # 페이지를 맨 아래까지 스크롤 다운
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    a_tag = soup.find('a', class_='productList_title__R1qZP linkAnchor')
    if a_tag and a_tag.get('href'):
        return urljoin(url, a_tag['href'])
    return None

# 사용할 URL
url = 'https://search.shopping.naver.com/catalog/35706026619?query=%ED%95%B8%EB%93%9C%ED%81%AC%EB%A6%BC&NaPm=ct%3Dlv2cgh88%7Cci%3Ddaf9de7fcf9c4f2a26d3b1c51bf3692ea08c4d2d%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D61d1eef1b14660ba0760343496c2d1b16db6f026'

# 링크 수집
collected_links = collect_lowest_price_link(url)
print(collected_links)

# WebDriver 종료
driver.quit()


https://cr.shopping.naver.com/adcr.nhn?x=uWpvJ0H%2BTHt6AAL7dlvtC%2F%2F%2F%2Fw%3D%3DsjrtJ%2BotwSQ%2BpgyvlBzUzGWojsfHd5NmTOsNaVtEMbIxFi1lbxjPcqDzB8Fhw%2Buzs9ilsWyDRFVaHyBVBQw6ZkxqwcdqveP%2B0%2Fti8AAG37l4Uhd8pfgWTEF%2FbGn8W9fb8qAZj2tPOvKFwUJicQFzPY%2FQk%2F7gJxLtQNiMc2dZmeeGgC%2FNLSthwFtuaRV9HpOARkuzr%2BjXyjSYbr%2ByUK0k2evluAdMlZS8X2tOYxJUMluqVVdvHOlFWRGVstT7FxF3oJPFXyRZ68yY5YqoMEzWHo2nqptr895cIDSCHnizJcCF4hNUoWtlggj82lEruGfh4OYPg6TurJLp0gT1W%2FuFHo5iFrXPKsHCEFxBfV4XGjWerilkDeG%2B1rLU11UaeFf3disext5K2yYkF868rsMDqiSH%2FEmMY8daZ60TGJlr%2FbR9ndyN6C1E01MRC28P1SHzGY2v%2BZ0uTIJjQSqYRrYgvdA%2FdRetGcx4yhDPA5z6Xp%2FjtAr51dRKn9%2FbYSbienewMMqAwFEuX9E6hacQBc1rryShHewscezzF7pJdMs35CWbtcQzIdKUzASEl0BpdpfZyKfUbkggyu%2FpKbo4g23Tqv2A5vYDXylOYwjtpV1%2BsEGMOLbxhV7%2Fvy0PGcAaiIS0wKKBulPM%2FNMbocsIh%2FvBo1XP0712mJXSgSOj6ObQ70s9yDc13mDfw2R50LLGPPL44n%2FJrCJcd1%2BiK3U6Rmdhrp0wRVpgv3vlFjmZIVrX3NnTu9A6rJhnSU%2Fbku%2BrJ85Dm&nvMid=43994653820&catId=50000289


### 최저가링크, 리뷰 수집

작성자 : 장다은<br>
작성일 : 20240417

In [12]:
# '''
# 링크만 수집되고 리뷰가 수집이 안되는 코드
# 셀레니움이 켜지면 > 최저가 링크를 수집 후 링크에 한 번 방문하게 됨 > 다시 가격비교 url로 되돌아갈 수 없어서 리뷰수집을 같이 못하는것같음
# '''

# from selenium import webdriver
# import time
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin
# import pandas as pd

# # WebDriver 초기화
# driver = webdriver.Chrome()

# # 스크롤 다운 함수
# def scroll_down(driver):
#     last_height = driver.execute_script("return document.body.scrollHeight")
#     while True:
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(2)  # 페이지 로드를 위한 대기
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:  # 스크롤 높이가 변하지 않았다면 중단
#             break
#         last_height = new_height

# # 최저가 링크 수집 함수
# def collect_lowest_price_link(url):
#     driver.get(url)
#     scroll_down(driver)  # 페이지를 맨 아래까지 스크롤 다운
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     a_tag = soup.find('a', class_='productList_title__R1qZP linkAnchor')
#     if a_tag and a_tag.get('href'):
#         return urljoin(url, a_tag['href'])
#     return None

# # 리뷰 수집 함수
# def collect_reviews(url):
#     driver.get(url)
#     scroll_down(driver)
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     reviews = [p.text for p in soup.find_all('p', class_='reviewItems_text__XrSSf')]
#     return reviews

# # 사용할 URL
# url = 'https://search.shopping.naver.com/catalog/35706026619?query=%ED%95%B8%EB%93%9C%ED%81%AC%EB%A6%BC&NaPm=ct%3Dlv2cgh88%7Cci%3Ddaf9de7fcf9c4f2a26d3b1c51bf3692ea08c4d2d%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D61d1eef1b14660ba0760343496c2d1b16db6f026'

# # 최저가 링크 수집
# lowest_price_link = collect_lowest_price_link(url)
# print("Lowest Price Link:", lowest_price_link)

# # 리뷰 수집
# if lowest_price_link:
#     reviews = collect_reviews(lowest_price_link)
#     print("Collected Reviews:", reviews)

# # WebDriver 종료
# driver.quit()


### 리뷰만 수집(자동화x)

작성자 : 장다은<br>
작성일 : 20240417

페이징 방식<br>
1페이지만 스크래핑함

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd

# WebDriver 초기화
driver = webdriver.Chrome()

# 스크롤 다운 함수
def scroll_down(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 페이지 로드를 위한 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 스크롤 높이가 변하지 않았다면 중단
            break
        last_height = new_height

# 리뷰 수집 함수
def collect_reviews(url):
    driver.get(url)
    
    # "쇼핑몰리뷰" 탭 클릭
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.floatingTab_on__2FzR0 a[aria-selected="true"] strong'))
    )
    review_tab = driver.find_element(By.CSS_SELECTOR, 'li.floatingTab_on__2FzR0 a[aria-selected="true"] strong')
    if review_tab.text == "쇼핑몰리뷰":
        review_tab.click()
    
    # 스크롤 다운하여 리뷰 로드
    scroll_down(driver)
    
    # 리뷰 내용 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = [p.text for p in soup.select('div.reviewItems_review__DqLYb div.reviewItems_review_text__dq0kE p.reviewItems_text__XrSSf')]
    return reviews

# 사용할 URL
url = 'https://search.shopping.naver.com/catalog/35706026619?query=%ED%95%B8%EB%93%9C%ED%81%AC%EB%A6%BC&NaPm=ct%3Dlv343law%7Cci%3D12806ae9a2a7a82c715e3eb856ee36b4cffd6111%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Db747e2ba76476588566356397d75026202e717d6'

# 리뷰 수집
reviews = collect_reviews(url)
print("Collected Reviews:", reviews)

# WebDriver 종료
driver.quit()


Collected Reviews: ['탬버린즈 핸드크림이 하도 유명하고 좋다고 해서 남편이 선물해줬어요~ㅎㅎ 향은 제가 골라서 라레와 카모로 골랐는데.. 둘 다 엄청 마음에 들어요!!박스 옆면에 음각으로 새겨진 글씨까지 감성적..!다른 향들 중에서도 특히 카모가 BEST상품이길래 구매했어요ㅎㅎ(제니Pick!)첫향은 카모마일+우드향이 강하게 올라와서 시원한 느낌이었는데, 잔향은 포근한 머스크 향이라 따뜻한 호텔 침구 속에 누워있는 느낌?시원함과 따뜻함을 동시에 가진 느낌이라 신기했고, 무엇보다 향이 진짜진짜 고급스러워서 역시 BEST는 괜히 BEST가 아니구나~~했답니다ㅎㅎ다쓰면 꼭꼭꼭 재구매할 핸드크림!!! 그리고 사용법은 샤넬 핸드크림처럼 짜는 형식!같은 향으로 튜브형도 있던데 저는 통통한 이 모양이 맘에 들어서 이걸로 Pick했어요ㅎㅎ뭔가 동글동글 조약돌 같은게 넘 귀엽지않나요?그리고 이렇게 향이 좋은 핸드크림들은 다 뻑뻑하고 건조하던데.. 왠걸.. 이건 발림성이 대박이었어요!그리고 엄청 촉촉하고 또 끈적임은 없이 쏙~ 흡수되더라구요..!! 이제 다른 핸드크림은 못 쓸 것 같아요!!(우리남편 왜 이제 사준거야!!ㅋㅋ^^)가격도 적당해서 친구나 애인에게 선물로 줘도 넘 센스있는 선물이 될 것 같아요ㅎㅎ임신, 출산으로 힘들고 지친 손에 향기로운 핸드크림으로 촉촉한 위로가 되었어요ㅎㅎ제품 넘 맘에 들어서 상세리뷰는 저의 개인 블로그에도 올려두었으니 궁금하신분들은 한번씩 들러주셔도 좋아요^^https://blog.naver.com/cocokkomimom22/223023655462', '제니픽!  탬버린즈 카모 핸드크림을 구입했습니다! 기존의 유니크한 탬버린즈만의 향이 너무 좋아서 바디로션과 카모 핸드크림까지 구입! 독특하고 고급스러운 핸드크림 케이스뿐만아니라 중성적이면서도 머스크향이 은은하게 나는 향까지 만족! 남편과도 함께 사용할수있을것같아요. 다쓰고나서 다른 향도 구입하고싶어요. 후각에 민감해서 타향수의 짙은 향은 기피하는편인데, 탬버린즈 향수 핸드크림은 은은

### 리뷰데이터 전부 수집(자동화x)

작성자 : 장다은<br>
작성일 : 20240417

한페이지당 20개 리뷰데이터 수집 가능, 최신순으로 100개까지 리뷰데이터 본문을 수집<br>
한 번 수집 시에 시간 측정해보는것도 필요할듯

In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# WebDriver 초기화
d = webdriver.Chrome()

# URL로 이동
url = "https://search.shopping.naver.com/catalog/35706026619?query=%ED%95%B8%EB%93%9C%ED%81%AC%EB%A6%BC&NaPm=ct%3Dlv343law%7Cci%3D12806ae9a2a7a82c715e3eb856ee36b4cffd6111%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Db747e2ba76476588566356397d75026202e717d6"
d.get(url)

# 필요한 정보 저장을 위한 리스트 초기화
reviews = []
page = 1
cnt = 1

while True:
    print("페이지", page, "\n")
    time.sleep(2)  # 페이지 로드를 위한 대기
    j = 1
    while True:  # 한 페이지에 리뷰 여러 개
        try:
            review_xpath = f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[{j}]/div[2]/div[1]/p'
            review_element = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.XPATH, review_xpath)))
            review = review_element.text
            reviews.append(review)
            if j % 2 == 0:  # 화면에 2개씩 보이도록 스크롤
                d.execute_script("arguments[0].scrollIntoView(true);", review_element)
            j += 1
            print(cnt, review, "\n")
            cnt += 1
        except Exception as e:
            print("Review collection error on page:", page, "error:", e)
            break

    time.sleep(2)

    # 페이지 이동 로직
    try:
        # 다음 페이지 버튼 로직 간소화
        next_page_buttons = d.find_elements(By.CSS_SELECTOR, 'a.pagination_next__3_3ip')
        if next_page_buttons and page < 5:
            next_page_buttons[-1].click()
            page += 1
        elif page == 5:
            print("Reached page limit of 5.")
            break
        else:
            print("No more pages found.")
            break
    except Exception as e:
        print("Failed to navigate from page", page, "error:", e)
        break

# WebDriver 종료
d.quit()

# 수집된 리뷰 출력
print("Collected Reviews:", reviews)


페이지 1 

1 탬버린즈 핸드크림이 하도 유명하고 좋다고 해서 남편이 선물해줬어요~ㅎㅎ 향은 제가 골라서 라레와 카모로 골랐는데.. 둘 다 엄청 마음에 들어요!!
박스 옆면에 음각으로 새겨진 글씨까지 감성적..!

다른 향들 중에서도 특히 카모가 BEST상품이길래 구매했어요ㅎㅎ(제니Pick!)
시원한 느낌이었는데, 잔향은 포근한 머스크 향이라 따뜻한 호텔 침구 속에 누워있는 느낌?
시원함과 따뜻함을 동시에 가진 느낌이라 신기했고, 무엇보다 향이 진짜진짜 고급스러워서 역시 BEST는 괜히 BEST가 아니구나~~했답니다ㅎㅎ
다쓰면 꼭꼭꼭 재구매할 핸드크림!!!

그리고 사용법은 샤넬 핸드크림처럼 짜는 형식!
같은 향으로 튜브형도 있던데 ㅎㅎ
뭔가 동글동글 조약돌 같은게 넘 귀엽지않나요?

그리고 건조하던데.. 왠걸.. 이건 발림성이 대박이었어요!
그리고 엄청 촉촉하고 또 끈적임은 없이 쏙~ 흡수되더라구요..!! 이제 다른 핸드크림은 못 쓸 것 같아요!!
(우리남편 왜 이제 사준거야!!ㅋㅋ^^)

가격도 적당해서 ㅎㅎ

임신, 출산으로 힘들고 지친 손에 향기로운 핸드크림으로 촉촉한 위로가 되었어요ㅎㅎ
^^
https://blog.naver.com/cocokkomimom22/223023655462 

2 제니픽! 탬버린즈 카모 핸드크림을 구입했습니다! 기존의 유니크한 탬버린즈만의 향이 너무 좋아서 바디로션과 카모 핸드크림까지 구입! 독특하고 고급스러운 핸드크림 케이스뿐만아니라 중성적이면서도 머스크향이 은은하게 나는 향까지 만족! 남편과도 함께 사용할수있을것같아요. 다쓰고나서 다른 향도 구입하고싶어요. 후각에 민감해서 타향수의 짙은 향은 기피하는편인데, 탬버린즈 향수 핸드크림은 은은한 향이라 좋습니다. 샘플 향수까지 주셔서 다른 향도 느껴볼수있어서 좋습니다. 핸드크림 뿐만아니라 바디로션도 디자인적으로훌륭해서 욕실에 놔뒀을때 인테리어 감성까지 놓치지않는 탬버린즈! 사용할때마다 작은 소확행처럼 나에게 투자하는 느낌이들어 기분좋아요 :) 

3 선물받고 탬버린즈 알게 됏어